In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import imageio
import cv2
import os
import tensorflow as tf
import math
import mediapipe as mp

from imutils import paths
from tensorflow_docs.vis import embed
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input
from time import time

In [ ]:
dataset_path = os.listdir('dataset/test')
room_types = os.listdir('dataset/test')

In [ ]:
rooms = []
for item in dataset_path:
    # 모든 파일 이름 가져오기
    all_rooms = os.listdir('dataset/train'+'/'+item)    
    # 리스트에 더하기
    for room in all_rooms:
        rooms.append((item, str('dataset/train'+'/'+item)+'/'+room))
# Build a dataframe
train_df = pd.DataFrame(data=rooms, columns=['tag','video_name'])
df = train_df.loc[:,['video_name','tag']]
df.to_csv('train.csv', encoding='utf-8-sig')

In [ ]:
rooms = []
for item in dataset_path:
    # 모든 파일 이름 가져오기
    all_rooms = os.listdir('dataset/test'+'/'+item)    
    # 리스트에 더하기
    for room in all_rooms:
        rooms.append((item, str('dataset/test'+'/'+item)+'/'+room))
# Build a dataframe
train_df = pd.DataFrame(data=rooms, columns=['tag','video_name'])
df = train_df.loc[:,['video_name','tag']]
df.to_csv('test.csv', encoding='utf-8-sig')

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0], 
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)]
        )
    except RuntimeError as e:
        print(e)

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
IMG_SIZE = 224

# mediapipe pose class를 초기화
mp_pose = mp.solutions.pose
# pose detect function에 image detect=True, 최소감지신뢰도 = 0.3, 모델 복잡도 =2를 준다.
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.3, model_complexity=2) 
# mediapipe의 drawing class를 초기화
mp_drawing = mp.solutions.drawing_utils

In [ ]:
def crop_center_square(frame):
    y,x = frame.shape[0:2]
    min_dim = min(y,x)
    start_x = (x//2)-(min_dim//2)
    start_y = (y//2)-(min_dim//2)
    return frame[start_y : start_y +min_dim, start_x:start_x+min_dim]

def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)  
            frame = cv2.resize(frame, resize)
            frame = frame[:,:,[2,1,0]]
            frames.append(frame)
            
            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)